# 한국어 언어모델 학습 및 다중 과제 튜닝
## KoGPT-2 기반의 챗봇 실습

> 작성자      
```
* 김성현 (bananaband657@gmail.com)  
김바다 (qkek983@gmail.com)
박상희 (parksanghee0103@gmail.com)  
이정우 (jungwoo.l2.rs@gmail.com)
```
[CC BY-NC-ND](https://creativecommons.org/licenses/by-nc-nd/2.0/kr/)

In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!apt-get install git-lfs

Detected operating system as Ubuntu/bionic.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 46 not upgraded.
Need to get 6,540 kB of archives.
After this operation, 15.3 MB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu bionic/main amd64 git-lfs amd64 2.13.2 [6,540 kB]
Fetched 6,540 kB in 2s (4,086 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 160979 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.13.2_a

In [ ]:
!git lfs install
!git clone https://huggingface.co/taeminlee/kogpt2

Git LFS initialized.
Cloning into 'kogpt2'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 43 (delta 16), reused 0 (delta 0)
Unpacking objects: 100% (43/43), done.


In [ ]:
!pip install transformers

     |████████████████████████████████| 1.9MB 12.3MB/s 
     |████████████████████████████████| 3.2MB 56.9MB/s 
     |████████████████████████████████| 890kB 65.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=4b137efc0948143223f766c27e3b6505a6a4d1b5c0566e63a424328643e14eab
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch
from tokenizers import SentencePieceBPETokenizer
from transformers import GPT2Config, GPT2LMHeadModel

tokenizer = SentencePieceBPETokenizer("/content/kogpt2/vocab.json", "/content/kogpt2/merges.txt")

config = GPT2Config(vocab_size=50000)
model = GPT2LMHeadModel(config)

model_dir = '/content/kogpt2/pytorch_model.bin'

model.load_state_dict(torch.load(model_dir, map_location='cuda'), strict=False)
model.to('cuda')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [ ]:
tokenized_text = tokenizer.encode('이순신은 조선 중기의 무신이다.', add_special_tokens=True)
print(tokenized_text)
print(tokenized_text.tokens)
print(tokenized_text.ids)

Encoding(num_tokens=7, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['▁이순', '신은', '▁조선', '▁중기의', '▁무신', '이다', '.']
[10925, 6647, 1117, 40249, 39793, 128, 47440]


In [ ]:
import torch
torch.manual_seed(42)

input_ids = torch.tensor(tokenizer.encode("이순신은", add_special_tokens=True).ids).unsqueeze(0).to('cuda')

output_sequences = model.generate(input_ids=input_ids, do_sample=True, max_length=100, num_return_sequences=3)
for generated_sequence in output_sequences:
    generated_sequence = generated_sequence.tolist()
    print("GENERATED SEQUENCE : {0}".format(tokenizer.decode(generated_sequence, skip_special_tokens=True)))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


GENERATED SEQUENCE : 이순신은 백옥담(이종원 분)의 시선으로 백성을 살피는 것을 알게 되었고, 천하에 대한 소신을 가지게 되었다.</s><s> 또한 성읍인 성전은 백옥담의 동생인 윤(尹)이 성전과 함께 있었는데 윤을 그의 아버지라고 불렀다.</s><s> 또한 성전이 완공될 때까지 성전에서 계속 놀 수 있었다.</s><s> 성전 안에서는 두 권의 도서가 있고, 이 문서에는 윤과 성국의 성전의 위치가 찍혀져 있다.
GENERATED SEQUENCE : 이순신은 이미 태연에게 마음을 돌린 상태.</s><s> 이어 "자칫 북한이 미사일 도발에 나설 수도 있다는 경고가 되고 있는 만큼 한미가 이에 대해 적절히 대응해주길 바란다"고 당부했다.</s><s> 또한 "오늘 김관진 제1위원장이 오랫만에 공개토론을 한 뒤 북한 도발을 중지하고 대화를 시작해야 한다고 강조한 것을 감안할 때 이번 주 미측과 한반도에서의 대화 분위기가 계속될 것으로 확신한다"고 말했다.</s><s> 또한 김 제1위원장의 이번 방문에서 미국측에 한반도 신뢰프로세스 차원의
GENERATED SEQUENCE : 이순신은 고려 태조 이성계를 도와 이성계 형진을 돕고 북경을 약탈케 했다.</s><s> 따라서, 그의 아들들 중에는 아누비스의 딸에게 왕위를 넘겨준 후(그녀에게 왕위를 주지 못한 후)나, 아버지에게 버림받은 후(그녀)가 왕좌에 오를 수 있었던 것을 보아야만 했다.</s><s> 1017년경, 아버지 레오나르가 사보이 왕국을 지배하게 되면서, 그의 후계자들은 모국(바빌론 왕국과 알


## 데이터셋 준비

In [ ]:
!git clone https://github.com/songys/Chatbot_data.git

Cloning into 'Chatbot_data'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 42 (delta 14), reused 0 (delta 0), pack-reused 18
Unpacking objects: 100% (42/42), done.


In [ ]:
import pandas as pd
data = pd.read_csv('/content/Chatbot_data/ChatbotData.csv')

In [ ]:
data.head(10)

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
5,SD카드 망가졌어,다시 새로 사는 게 마음 편해요.,0
6,SD카드 안돼,다시 새로 사는 게 마음 편해요.,0
7,SNS 맞팔 왜 안하지ㅠㅠ,잘 모르고 있을 수도 있어요.,0
8,SNS 시간낭비인 거 아는데 매일 하는 중,시간을 정하고 해보세요.,0
9,SNS 시간낭비인데 자꾸 보게됨,시간을 정하고 해보세요.,0


In [ ]:
added_special_token_num = tokenizer.add_special_tokens(['<s>', '</s>'])
print(added_special_token_num)

0


In [ ]:
pad_id = tokenizer.token_to_id("<pad>")
print(pad_id)
tokenizer.enable_padding(pad_id=pad_id, pad_token="<pad>")
tokenizer.enable_truncation(max_length=128)

3


In [ ]:
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, file_path):
        self.data = []
        self.file_path = file_path
        self.tokenizer = tokenizer
    
    def load_data(self):
        raw_data = pd.read_csv(self.file_path)
        train_data = '<s>'+raw_data['Q']+'</s>'+'<s>'+raw_data['A']+'</s>'
        #<s>안녕하세요</s><s> -> 네, 안녕하세요</s>
        tokenized_train_data = tokenizer.encode_batch(train_data)
        for single_data in tokenized_train_data:
            self.data.append(torch.tensor(single_data.ids).unsqueeze(0))
                
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        item = self.data[index]
        return item

In [ ]:
train_dataset = ChatDataset(tokenizer=tokenizer, file_path='/content/Chatbot_data/ChatbotData.csv')
train_dataset.load_data()

In [ ]:
from torch.utils.data import DataLoader

data_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

In [ ]:
from transformers import AdamW

In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-4, correct_bias=True)

epochs = 3

avg_loss = (0.0, 0.0)
for epoch in range(epochs):
    count=0
    for data in data_loader:
        optimizer.zero_grad()
        data = data.transpose(1,0)
        data = data.to('cuda')
        model = model.to('cuda')
        
        outputs = model(data, labels=data)
        loss, logits = outputs[:2]
        loss = loss.to('cuda')
        loss.backward()
        avg_loss = (avg_loss[0] * 0.99 + loss, avg_loss[1] * 0.99 + 1.0)
        optimizer.step()
        if count % 200 == 0:
            print('epoch no.{0}  train ({1}/{2})  loss = {3:.5f}  avg_loss = {4:.5f}' . format(epoch, count, len(data_loader), loss, avg_loss[0] / avg_loss[1]))
        count += 1


epoch no.0  train (0/2956)  loss = 2.60970  avg_loss = 2.60970
epoch no.0  train (200/2956)  loss = 1.71297  avg_loss = 1.44216
epoch no.0  train (400/2956)  loss = 0.93555  avg_loss = 1.29001
epoch no.0  train (600/2956)  loss = 1.07984  avg_loss = 1.21623
epoch no.0  train (800/2956)  loss = 1.33416  avg_loss = 1.20830
epoch no.0  train (1000/2956)  loss = 1.57980  avg_loss = 1.19345
epoch no.0  train (1200/2956)  loss = 1.20132  avg_loss = 1.18999
epoch no.0  train (1400/2956)  loss = 0.86411  avg_loss = 1.15901
epoch no.0  train (1600/2956)  loss = 1.06986  avg_loss = 1.14346
epoch no.0  train (1800/2956)  loss = 1.17722  avg_loss = 1.14398
epoch no.0  train (2000/2956)  loss = 0.91722  avg_loss = 1.17108
epoch no.0  train (2200/2956)  loss = 1.05489  avg_loss = 1.12717
epoch no.0  train (2400/2956)  loss = 0.77489  avg_loss = 1.13079
epoch no.0  train (2600/2956)  loss = 1.10603  avg_loss = 1.15694
epoch no.0  train (2800/2956)  loss = 1.24256  avg_loss = 1.12590
epoch no.1  train

In [ ]:
torch.save(model.state_dict(), 'chitchat_model.bin')

In [ ]:
def encoding(text):
    text = '<s>'+text+'</s><s>'
    return torch.tensor(tokenizer.encode(text).ids).unsqueeze(0).to('cuda')

def decoding(ids):
    return tokenizer.decode_batch(ids)

tokenizer.no_padding()
tokenizer.no_truncation()

e_s = tokenizer.token_to_id('</s>')
unk = tokenizer.token_to_id('<unk>')

In [ ]:
def get_answer(input_sent):
    input_ids = encoding(input_sent)

    sample_outputs = model.generate(
        input_ids,
        num_return_sequences=5,
        do_sample=True, 
        max_length=128, 
        top_k=50, 
        top_p=0.95, 
        eos_token_id=e_s,
        early_stopping=True,
        bad_words_ids=[[unk]]
    )

    decoded_result = decoding(sample_outputs.tolist())
    for result in decoded_result:
        print(result)

In [ ]:
get_answer('안녕?')

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


안녕? 아직 많이 부족한가봐요.
안녕? 안녕하세요.
안녕? 안녕하세요.
안녕? 안녕하세요.
안녕? 안녕.


In [ ]:
get_answer('만나서 반가워.')

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


만나서 반가워. 친구라고 생각하나봐요.
만나서 반가워. 저도 만나고 싶어요.
만나서 반가워. 이제서야 반가워하게 되었어요.
만나서 반가워. 오랜만에 보는 자리라 그런가봐요.
만나서 반가워. 좋은 사람 많이 만나세요.


In [ ]:
get_answer('인공지능의 미래에 대해 어떻게 생각하세요?')

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


인공지능의 미래에 대해 어떻게 생각하세요? 인공지능이 발전할 거예요.
인공지능의 미래에 대해 어떻게 생각하세요? 인공지능의 미래는 예측하기 힘들어요.
인공지능의 미래에 대해 어떻게 생각하세요? 인공지능이 탄생할거예요. 인공지능을 잘 이용하면 다른 사람의 눈을 더 많이 볼 수 있대요.
인공지능의 미래에 대해 어떻게 생각하세요? 인공지능이 뭐든 뭐든 관심이 있어요. 인공지능에 대한 지식이 많으면 좋겠어요.
인공지능의 미래에 대해 어떻게 생각하세요? 인공지능이 현실을 너무 초월하고 초월적인 것은 피할 수 있어도 큰 변화이겠지요.


In [ ]:
get_answer('여자친구 선물 추천해줘.')

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


여자친구 선물 추천해줘. 같이 여행을 떠나자고 말해보세요.
여자친구 선물 추천해줘. 현금이 바닥난 것 같아요.
여자친구 선물 추천해줘. 직접 물어보세요.
여자친구 선물 추천해줘. 네 말씀해주세요.
여자친구 선물 추천해줘. 당신이 생각하는 선물 고르세요.


In [ ]:
get_answer('앞으로 인공지능이 어떻게 발전하게 될까요?')

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


앞으로 인공지능이 어떻게 발전하게 될까요? 인공지능은 무궁무진하니까요.
앞으로 인공지능이 어떻게 발전하게 될까요? 사람을 변덕스럽게 만들어요. 인공지능은 발전 가능성이 가장 높겠지요.
앞으로 인공지능이 어떻게 발전하게 될까요? 인공지능 발전이 생각보다 빠르게 와닿더라고요.
앞으로 인공지능이 어떻게 발전하게 될까요? 인공지능은 기술이 될 수 있어요.
앞으로 인공지능이 어떻게 발전하게 될까요? 사람과 기계에 대한 이해가 시급하네요. 인공지능에 대한 관심이 높아지기 때문에 쉽지 않을거예요. 인공지능을 이해하면 금방 풀릴 거예요


In [ ]:
get_answer('이제 그만 수업 끝내자.')

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


이제 그만 수업 끝내자. 다음 학기에는 할 수 있을 거예요.
이제 그만 수업 끝내자. 더 좋은 학원생활 할 수 있을 거예요.
이제 그만 수업 끝내자. 잘했어요. 잘할 수 있을 거예요.
이제 그만 수업 끝내자. 저도 많이 배웠어요.
이제 그만 수업 끝내자. 공부는 끝이 없어요.
